In [ ]:
%load_ext autoreload
%autoreload 2

from model import RetrieverModel
from datasets import load_dataset

# loss = "siglip"
loss = "clip"

model = RetrieverModel(loss_type=loss)
dataset = load_dataset("microsoft/ms_marco", "v1.1")

In [ ]:
batch_size = 32
# epochs = 2 * batch_size
epochs = 10
lr = 1e-6
eval_batch = 1000
power = 1

data_path = "training"
path = "data/{}.jsonl"
pubmed = path.format(data_path)
TAR = path.format("TAR_data")
sysrev = path.format("sysrev_conv")
train_sources = ['pubmed-searchrefiner']
# train_sources += ['pubmed-query', 'raw-jsonl']

print(dataset)
lr_n = "" if lr == 1e-7 else f"lr{lr:.0E}_"
b_n = "" if batch_size == 2 else f"b{batch_size}_"
pow_n = "" if power == 1 else f"^{power}"

model_name = model.model_name.split("/")[-1]
model_path = f"{loss}/{model_name}/{b_n + lr_n}{pow_n}"
print(model_path)

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})
clip/SPLADE-BERT-Mini/b32_lr1E-05_


In [ ]:
import os
from transformers import Trainer, TrainingArguments
from callbacks import NotebookProgressCallbackNoTable
from transformers.utils.notebook import NotebookProgressCallback
from evaluation import compute_metrics

os.environ["WANDB_PROJECT"] = "MSE"
os.environ["WANDB_LOG_MODEL"] = "false"

# epochs = 10
training_args = TrainingArguments(
    output_dir="models/" + model_path,
    per_device_train_batch_size=batch_size,
    num_train_epochs=epochs,
    learning_rate=lr,
    save_steps=1000,
    save_total_limit=1,
    remove_unused_columns=False,
    bf16=True,
    optim="adamw_bnb_8bit",
    logging_steps=100,
    eval_steps=200,
    eval_strategy="steps",
    eval_on_start=True,
    per_device_eval_batch_size=eval_batch,
    run_name=model_path,
    report_to='wandb',
    # max_steps=1000,
)

def collate_fn(batch):
    return {
        "query": [ex["query"] for ex in batch],
        "answer": [ex["answers"][0] if ex["answers"] else "" for ex in batch],
    }

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

trainer.remove_callback(NotebookProgressCallback)
trainer.add_callback(NotebookProgressCallbackNoTable)

# trainer.evaluate()
trainer.train()
# try: trainer.train(resume_from_checkpoint=True)
# except: trainer.train(resume_from_checkpoint=False)

wandb: Currently logged in as: simon-doehl (simon-doehl-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
